In [ ]:
%cd "full/path/to/codebase"
import base64
import os
from io import BytesIO
from pathlib import Path

import polars as pl
from dotenv import load_dotenv
from openai import OpenAI
from PIL import Image
from pydantic import BaseModel
from qdrant_client.models import FieldCondition, Filter, MatchValue, Prefetch
from tqdm import tqdm

from src.data.dataset import LawCorpus, TestDataset
from src.db.qdrant import Qdrant
from src.nn.text_embedding import TextEmbedding

load_dotenv()

/home/anhvd27/miniforge3/envs/mlqa-tsr/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/anhvd27/miniforge3/envs/mlqa-tsr/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/anhvd27/projects/vlsp/mlqa-tsr/codebase


/home/anhvd27/miniforge3/envs/mlqa-tsr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
text_embedding_model = TextEmbedding()

flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn i

In [3]:
def resize_image(
    image: Image.Image,
    max_height: int = 1536,
    max_width: int = 1536,
):
    if image.width <= max_width and image.height <= max_height:
        return image

    if image.width >= image.height:
        new_width = max_width
        new_height = new_width * image.height // image.width
        image.resize((new_width, new_height), Image.Resampling.LANCZOS)
        return image

    if image.height >= image.width:
        new_height = max_height
        new_width = new_height * image.width // image.height
        image.resize((new_width, new_height), Image.Resampling.LANCZOS)
        return image

    return image


def encode_image(image_path: str):
    image = Image.open(image_path).convert("RGB")
    image = resize_image(image)
    image_buffer = BytesIO()
    image.save(image_buffer, format="JPEG")
    return base64.b64encode(image_buffer.getvalue()).decode("utf-8")

In [ ]:
openai_client = OpenAI(
    api_key=os.getenv("VLLM_API_KEY"),
    base_url=os.getenv("VLLM_BASE_URL"),
)
model_name = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"


class Result(BaseModel):
    choice: str
    # explanation: str


system_prompt = """ Given an image and a question both about traffic in Vietnam,
Multiple choices and yes/no questions shall be provided,
If A, B, C, D were given, choose the letter only,
If Đúng; Sai were given, choose Đúng or Sai only,
No need explanation needed
"""

In [ ]:
qdrant = Qdrant()
law_corpus = LawCorpus(Path("../data/law_db"))
# path to private test folder, perhaps one need to slightly change the file names and image path
# read the code in src/data/dataset.py
test_set_2 = TestDataset(Path("../data/private_test"), law_corpus, task_number=2)
image_feature_df = pl.read_json("assets/private_test_image_feature.json")

In [6]:
def format_text(data_point: dict):
    fmt_text_input = "Question: " + data_point["question"] + "\nOptions:\n"
    if data_point["question_type"] == "Multiple choice":
        for k, v in data_point["choices"].items():
            fmt_text_input += f"{k}: {v} \n"
        default_choice = "A"
    elif data_point["question_type"] == "Yes/No":
        fmt_text_input += "Đúng\nSai\n"
        default_choice = "Đúng"
    fmt_text_input += "Choice: "
    if data_point.get("answer", ""):
        fmt_text_input += data_point["answer"]
    return fmt_text_input, default_choice

In [7]:
collection_name = "train_data"
for i in tqdm(range(len(test_set_2))):
    image_path = test_set_2[i]["image_path"]
    base64_image = encode_image(image_path)

    fmt_text_input, default_choice = format_text(test_set_2[i])

    image_feature_point = image_feature_df.filter(pl.col("image_name") == Path(image_path).name).row(0, named=True)
    image_object_feature_list_vector = [[0.0] * 2304]
    if image_feature_point["object_feature_list"]:
        image_object_feature_list_vector = [
            img_obj["object_feature"] for img_obj in image_feature_point["object_feature_list"]
        ]

    someshots = qdrant.client.query_points(
        collection_name=collection_name,
        prefetch=Prefetch(
            prefetch=Prefetch(
                query=text_embedding_model.infer_single(fmt_text_input),
                using="text_vector",
                limit=10,
                filter=Filter(
                    must=FieldCondition(key="question_type", match=MatchValue(value=test_set_2[i]["question_type"]))
                ),
            ),
            query=image_feature_point["general_feature"],
            using="image_general_feature_vector",
            limit=5,
        ),
        query=image_object_feature_list_vector,
        using="image_object_feature_list_vector",
        limit=3,
        with_payload=True,
    )

    messages = []
    messages.append(
        {
            "role": "system",
            "content": system_prompt,
        }
    )
    if someshots.points:
        for shot in someshots.points:
            if not shot.payload:
                continue
            shot_image_path = shot.payload["image_path"]
            messages.append(
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": format_text(shot.payload)[0]},
                        {
                            "type": "image_url",
                            "image_url": {"url": f"data:image/jpeg;base64,{encode_image(shot_image_path)}"},
                        },
                    ],
                }
            )
    messages.append(
        {
            "role": "user",
            "content": [
                {"type": "text", "text": fmt_text_input},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
            ],
        }
    )
    completion = openai_client.chat.completions.parse(
        model=model_name,
        messages=messages,
        response_format=Result,
    )
    model_result = completion.choices[0].message.parsed
    if model_result is None:
        test_set_2.index_result(index=i, answer=default_choice)
    if model_result:
        test_set_2.index_result(index=i, answer=model_result.choice)

100%|██████████| 146/146 [04:43<00:00,  1.95s/it]


In [8]:
test_set_2.save_results("assets/private_submissions/submission_task2.json")

True